# Import Dependencies

In [110]:
import requests
import json
import pandas as pd

from sqlalchemy import create_engine, ForeignKey, Column, String, Integer, Text, Boolean, Float
from sqlalchemy.ext.declarative import declarative_base

import mysql.connector

# Saved password in config file which will be gitignored
from config import pw

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

# Extract Pokemon Information

In [21]:
#Use the Pokemon Species APIs to populate lists
poke_name = []
poke_happy = []
poke_catch = []
poke_color = []
poke_evolve = []
poke_gender = []
poke_generation = []
poke_growth = []
poke_habitat = []
poke_id = []
poke_shape = []
poke_baby = []
poke_leg = []
poke_myth = []
    
for s in range(1000):
    url = "https://pokeapi.co/api/v2/pokemon-species/"+str(s+1)
    try:
        response = requests.get(url).json()
        poke_name.append(response["name"])
        poke_happy.append(response["base_happiness"])
        poke_catch.append(response["capture_rate"])
        poke_color.append(response["color"]["name"])
        poke_evolve.append(response["evolves_from_species"])
        poke_gender.append(response["gender_rate"])
        poke_generation.append(response["generation"]["name"])
        poke_growth.append(response["growth_rate"]["name"])
        try:
            poke_habitat.append(response["habitat"]["name"])
        except TypeError:
            poke_habitat.append("N/A")
        poke_id.append(response["id"])
        try:
            poke_shape.append(response["shape"]["name"])
        except TypeError:
            poke_shape.append("N/A")
        poke_baby.append(response["is_baby"])
        poke_leg.append(response["is_legendary"])
        poke_myth.append(response["is_mythical"])
    except json.decoder.JSONDecodeError:
        pass

In [22]:
# Use the populates lists to make a dictionary
pokemon_dict = {
    "id": poke_id,
    "name": poke_name,
    "generation": poke_generation,
    "color" : poke_color,
    "shape" : poke_shape,
    "gender_rate" : poke_gender,
    "growth_rate": poke_growth,
    "evolution": poke_evolve,
    "habitat": poke_habitat,
    "catch_rate": poke_catch,
    "base_happiness": poke_happy,
    "is_baby":poke_baby,
    "is_legendary": poke_leg,
    "is_mythical": poke_myth  
    
}

In [23]:
pokemon_dict

{'id': [1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  41,
  42,
  43,
  44,
  45,
  46,
  47,
  48,
  49,
  50,
  51,
  52,
  53,
  54,
  55,
  56,
  57,
  58,
  59,
  60,
  61,
  62,
  63,
  64,
  65,
  66,
  67,
  68,
  69,
  70,
  71,
  72,
  73,
  74,
  75,
  76,
  77,
  78,
  79,
  80,
  81,
  82,
  83,
  84,
  85,
  86,
  87,
  88,
  89,
  90,
  91,
  92,
  93,
  94,
  95,
  96,
  97,
  98,
  99,
  100,
  101,
  102,
  103,
  104,
  105,
  106,
  107,
  108,
  109,
  110,
  111,
  112,
  113,
  114,
  115,
  116,
  117,
  118,
  119,
  120,
  121,
  122,
  123,
  124,
  125,
  126,
  127,
  128,
  129,
  130,
  131,
  132,
  133,
  134,
  135,
  136,
  137,
  138,
  139,
  140,
  141,
  142,
  143,
  144,
  145,
  146,
  147,
  148,
  149,
  150,
  151,
  152,
  153,
  154,
  155,
  156,
  157,
  1

In [26]:
#Use the dictionary to make a DataFrame
poke_df = pd.DataFrame(pokemon_dict)
poke_df.head(50)

,id,name,generation,color,shape,gender_rate,growth_rate,evolution,habitat,catch_rate,base_happiness,is_baby,is_legendary,is_mythical
0,1,bulbasaur,generation-i,green,quadruped,1,medium-slow,None,grassland,45,50.0,False,False,False
1,2,ivysaur,generation-i,green,quadruped,1,medium-slow,"{'name': 'bulbasaur', 'url': 'https://pokeapi....",grassland,45,50.0,False,False,False
2,3,venusaur,generation-i,green,quadruped,1,medium-slow,"{'name': 'ivysaur', 'url': 'https://pokeapi.co...",grassland,45,50.0,False,False,False
3,4,charmander,generation-i,red,upright,1,medium-slow,None,mountain,45,50.0,False,False,False
4,5,charmeleon,generation-i,red,upright,1,medium-slow,"{'name': 'charmander', 'url': 'https://pokeapi...",mountain,45,50.0,False,False,False
5,6,charizard,generation-i,red,upright,1,medium-slow,"{'name': 'charmeleon', 'url': 'https://pokeapi...",mountain,45,50.0,False,False,False
6,7,squirtle,generation-i,blue,upright,1,medium-slow,None,waters-edge,45,50.0,False,False,False
7,8,wartortle,generation-i,blue,upright,1,medium-slow,"{'name': 'squirtle', 'url': 'https://pokeapi.c...",waters-edge,45,50.0,False,False,False
8,9,blastoise,generation-i,blue,upright,1,medium-slow,"{'name': 'wartortle', 'url': 'https://pokeapi....",waters-edge,45,50.0,False,False,False
9,10,caterpie,generation-i,green,armor,4,medium,None,forest,255,50.0,False,False,False


In [27]:
#Use the Pokemon API to populate additional lists 
poke_id2 = []
poke_ability = []
poke_exp = []
poke_height = []
poke_sprite = []
poke_shiny = []
poke_hp = []
poke_attack = []
poke_def = []
poke_spatk = []
poke_spdef = []
poke_speed = []
poke_type1 = []
poke_type2 = []
poke_weight = []

for p in range(1000):
    url = "https://pokeapi.co/api/v2/pokemon/"+str(p+1)
    response = requests.get(url).json()
    poke_id2.append(response["id"])
    poke_ability.append(response["abilities"][0]["ability"]["name"])
    poke_exp.append(response["base_experience"])
    poke_height.append(response["height"])
    poke_sprite.append(response["sprites"]["front_default"])
    poke_shiny.append(response["sprites"]["front_shiny"])
    poke_hp.append(response["stats"][0]["base_stat"])
    poke_attack.append(response["stats"][1]["base_stat"])
    poke_def.append(response["stats"][2]["base_stat"])
    poke_spatk.append(response["stats"][3]["base_stat"])
    poke_spdef.append(response["stats"][4]["base_stat"])
    poke_speed.append(response["stats"][5]["base_stat"])
    poke_type1.append(response["types"])
    poke_weight.append(response["weight"])

In [28]:
# Use the populates lists to make a dictionary
poke_dict2 = {
    "id": poke_id2,
    "ability": poke_ability,
    "base_xp": poke_exp,
    "height": poke_height,
    "weight": poke_weight,
    "standard_pic": poke_sprite,
    "shiny_pic": poke_shiny,
    "base_hp": poke_hp,
    "base_attack": poke_attack,
    "base_def": poke_def,
    "base_sp_attack": poke_spatk,
    "base_sp_def": poke_spdef,
    "base_speed" : poke_speed,
    "type_1" : poke_type1,
    #"type_2" : poke_type2
    }

In [29]:
#Use the dictionary to make a DataFrame
poke_df2 = pd.DataFrame(poke_dict2)
poke_df2

,id,ability,base_xp,height,weight,standard_pic,shiny_pic,base_hp,base_attack,base_def,base_sp_attack,base_sp_def,base_speed,type_1
0,1,overgrow,64.0,7,69,https://raw.githubusercontent.com/PokeAPI/spri...,https://raw.githubusercontent.com/PokeAPI/spri...,45,49,49,65,65,45,"[{'slot': 1, 'type': {'name': 'grass', 'url': ..."
1,2,overgrow,142.0,10,130,https://raw.githubusercontent.com/PokeAPI/spri...,https://raw.githubusercontent.com/PokeAPI/spri...,60,62,63,80,80,60,"[{'slot': 1, 'type': {'name': 'grass', 'url': ..."
2,3,overgrow,263.0,20,1000,https://raw.githubusercontent.com/PokeAPI/spri...,https://raw.githubusercontent.com/PokeAPI/spri...,80,82,83,100,100,80,"[{'slot': 1, 'type': {'name': 'grass', 'url': ..."
3,4,blaze,62.0,6,85,https://raw.githubusercontent.com/PokeAPI/spri...,https://raw.githubusercontent.com/PokeAPI/spri...,39,52,43,60,50,65,"[{'slot': 1, 'type': {'name': 'fire', 'url': '..."
4,5,blaze,142.0,11,190,https://raw.githubusercontent.com/PokeAPI/spri...,https://raw.githubusercontent.com/PokeAPI/spri...,58,64,58,80,65,80,"[{'slot': 1, 'type': {'name': 'fire', 'url': '..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,996,thermal-exchange,NaN,5,170,https://raw.githubusercontent.com/PokeAPI/spri...,None,65,75,45,35,45,55,"[{'slot': 1, 'type': {'name': 'dragon', 'url':..."
996,997,thermal-exchange,NaN,8,300,https://raw.githubusercontent.com/PokeAPI/spri...,None,90,95,66,45,65,62,"[{'slot': 1, 'type': {'name': 'dragon', 'url':..."
997,998,thermal-exchange,NaN,21,2100,https://raw.githubusercontent.com/PokeAPI/spri...,None,115,145,92,75,86,87,"[{'slot': 1, 'type': {'name': 'dragon', 'url':..."
998,999,rattled,NaN,3,50,https://raw.githubusercontent.com/PokeAPI/spri...,None,45,30,70,75,70,10,"[{'slot': 1, 'type': {'name': 'ghost', 'url': ..."


In [30]:
#Merge the two Dataframes
poke_merge = pd.merge(poke_df, poke_df2, left_index=True, right_index=True)
poke_merge.head()

,id_x,name,generation,color,shape,gender_rate,growth_rate,evolution,habitat,catch_rate,...,weight,standard_pic,shiny_pic,base_hp,base_attack,base_def,base_sp_attack,base_sp_def,base_speed,type_1
0,1,bulbasaur,generation-i,green,quadruped,1,medium-slow,None,grassland,45,...,69,https://raw.githubusercontent.com/PokeAPI/spri...,https://raw.githubusercontent.com/PokeAPI/spri...,45,49,49,65,65,45,"[{'slot': 1, 'type': {'name': 'grass', 'url': ..."
1,2,ivysaur,generation-i,green,quadruped,1,medium-slow,"{'name': 'bulbasaur', 'url': 'https://pokeapi....",grassland,45,...,130,https://raw.githubusercontent.com/PokeAPI/spri...,https://raw.githubusercontent.com/PokeAPI/spri...,60,62,63,80,80,60,"[{'slot': 1, 'type': {'name': 'grass', 'url': ..."
2,3,venusaur,generation-i,green,quadruped,1,medium-slow,"{'name': 'ivysaur', 'url': 'https://pokeapi.co...",grassland,45,...,1000,https://raw.githubusercontent.com/PokeAPI/spri...,https://raw.githubusercontent.com/PokeAPI/spri...,80,82,83,100,100,80,"[{'slot': 1, 'type': {'name': 'grass', 'url': ..."
3,4,charmander,generation-i,red,upright,1,medium-slow,None,mountain,45,...,85,https://raw.githubusercontent.com/PokeAPI/spri...,https://raw.githubusercontent.com/PokeAPI/spri...,39,52,43,60,50,65,"[{'slot': 1, 'type': {'name': 'fire', 'url': '..."
4,5,charmeleon,generation-i,red,upright,1,medium-slow,"{'name': 'charmander', 'url': 'https://pokeapi...",mountain,45,...,190,https://raw.githubusercontent.com/PokeAPI/spri...,https://raw.githubusercontent.com/PokeAPI/spri...,58,64,58,80,65,80,"[{'slot': 1, 'type': {'name': 'fire', 'url': '..."


In [31]:
#Sepearate the Dictionary in the Evolution Colum
df_evo_clean = pd.json_normalize(poke_merge['evolution'])

#Merge this to dataframe on the index
df_evo_1 = pd.merge(poke_merge, df_evo_clean, left_index=True, right_index=True)
df_evo_1.head()

,id_x,name_x,generation,color,shape,gender_rate,growth_rate,evolution,habitat,catch_rate,...,shiny_pic,base_hp,base_attack,base_def,base_sp_attack,base_sp_def,base_speed,type_1,name_y,url
0,1,bulbasaur,generation-i,green,quadruped,1,medium-slow,None,grassland,45,...,https://raw.githubusercontent.com/PokeAPI/spri...,45,49,49,65,65,45,"[{'slot': 1, 'type': {'name': 'grass', 'url': ...",NaN,NaN
1,2,ivysaur,generation-i,green,quadruped,1,medium-slow,"{'name': 'bulbasaur', 'url': 'https://pokeapi....",grassland,45,...,https://raw.githubusercontent.com/PokeAPI/spri...,60,62,63,80,80,60,"[{'slot': 1, 'type': {'name': 'grass', 'url': ...",bulbasaur,https://pokeapi.co/api/v2/pokemon-species/1/
2,3,venusaur,generation-i,green,quadruped,1,medium-slow,"{'name': 'ivysaur', 'url': 'https://pokeapi.co...",grassland,45,...,https://raw.githubusercontent.com/PokeAPI/spri...,80,82,83,100,100,80,"[{'slot': 1, 'type': {'name': 'grass', 'url': ...",ivysaur,https://pokeapi.co/api/v2/pokemon-species/2/
3,4,charmander,generation-i,red,upright,1,medium-slow,None,mountain,45,...,https://raw.githubusercontent.com/PokeAPI/spri...,39,52,43,60,50,65,"[{'slot': 1, 'type': {'name': 'fire', 'url': '...",NaN,NaN
4,5,charmeleon,generation-i,red,upright,1,medium-slow,"{'name': 'charmander', 'url': 'https://pokeapi...",mountain,45,...,https://raw.githubusercontent.com/PokeAPI/spri...,58,64,58,80,65,80,"[{'slot': 1, 'type': {'name': 'fire', 'url': '...",charmander,https://pokeapi.co/api/v2/pokemon-species/4/


In [32]:
#Sepearate the Dictionary in the type Colum
df_type_clean = pd.json_normalize(poke_merge['type_1'])
df_type_clean2 = pd.json_normalize(df_type_clean[1])
df_type_clean1 = pd.json_normalize(df_type_clean[0])
types_df = pd.merge(df_type_clean1,df_type_clean2,left_index=True, right_index=True)
types_df.head()

,slot_x,type.name_x,type.url_x,slot_y,type.name_y,type.url_y
0,1,grass,https://pokeapi.co/api/v2/type/12/,2.0,poison,https://pokeapi.co/api/v2/type/4/
1,1,grass,https://pokeapi.co/api/v2/type/12/,2.0,poison,https://pokeapi.co/api/v2/type/4/
2,1,grass,https://pokeapi.co/api/v2/type/12/,2.0,poison,https://pokeapi.co/api/v2/type/4/
3,1,fire,https://pokeapi.co/api/v2/type/10/,NaN,NaN,NaN
4,1,fire,https://pokeapi.co/api/v2/type/10/,NaN,NaN,NaN


In [33]:
#Rename the desired columns
types_df1 = types_df.rename(columns={'type.name_x': 'type_1', 'type.name_y':'type_2'})
types_df1.head()

,slot_x,type_1,type.url_x,slot_y,type_2,type.url_y
0,1,grass,https://pokeapi.co/api/v2/type/12/,2.0,poison,https://pokeapi.co/api/v2/type/4/
1,1,grass,https://pokeapi.co/api/v2/type/12/,2.0,poison,https://pokeapi.co/api/v2/type/4/
2,1,grass,https://pokeapi.co/api/v2/type/12/,2.0,poison,https://pokeapi.co/api/v2/type/4/
3,1,fire,https://pokeapi.co/api/v2/type/10/,NaN,NaN,NaN
4,1,fire,https://pokeapi.co/api/v2/type/10/,NaN,NaN,NaN


In [34]:
#Merge the cleaned types
df_merge_2=pd.merge(df_evo_1,types_df1,left_index=True, right_index=True)
df_merge_2.head()

,id_x,name_x,generation,color,shape,gender_rate,growth_rate,evolution,habitat,catch_rate,...,base_speed,type_1_x,name_y,url,slot_x,type_1_y,type.url_x,slot_y,type_2,type.url_y
0,1,bulbasaur,generation-i,green,quadruped,1,medium-slow,None,grassland,45,...,45,"[{'slot': 1, 'type': {'name': 'grass', 'url': ...",NaN,NaN,1,grass,https://pokeapi.co/api/v2/type/12/,2.0,poison,https://pokeapi.co/api/v2/type/4/
1,2,ivysaur,generation-i,green,quadruped,1,medium-slow,"{'name': 'bulbasaur', 'url': 'https://pokeapi....",grassland,45,...,60,"[{'slot': 1, 'type': {'name': 'grass', 'url': ...",bulbasaur,https://pokeapi.co/api/v2/pokemon-species/1/,1,grass,https://pokeapi.co/api/v2/type/12/,2.0,poison,https://pokeapi.co/api/v2/type/4/
2,3,venusaur,generation-i,green,quadruped,1,medium-slow,"{'name': 'ivysaur', 'url': 'https://pokeapi.co...",grassland,45,...,80,"[{'slot': 1, 'type': {'name': 'grass', 'url': ...",ivysaur,https://pokeapi.co/api/v2/pokemon-species/2/,1,grass,https://pokeapi.co/api/v2/type/12/,2.0,poison,https://pokeapi.co/api/v2/type/4/
3,4,charmander,generation-i,red,upright,1,medium-slow,None,mountain,45,...,65,"[{'slot': 1, 'type': {'name': 'fire', 'url': '...",NaN,NaN,1,fire,https://pokeapi.co/api/v2/type/10/,NaN,NaN,NaN
4,5,charmeleon,generation-i,red,upright,1,medium-slow,"{'name': 'charmander', 'url': 'https://pokeapi...",mountain,45,...,80,"[{'slot': 1, 'type': {'name': 'fire', 'url': '...",charmander,https://pokeapi.co/api/v2/pokemon-species/4/,1,fire,https://pokeapi.co/api/v2/type/10/,NaN,NaN,NaN


In [35]:
#Check the Colums in the DataFrame to identify the ones you want to keep
print(df_merge_2.columns.tolist())

['id_x', 'name_x', 'generation', 'color', 'shape', 'gender_rate', 'growth_rate', 'evolution', 'habitat', 'catch_rate', 'base_happiness', 'is_baby', 'is_legendary', 'is_mythical', 'id_y', 'ability', 'base_xp', 'height', 'weight', 'standard_pic', 'shiny_pic', 'base_hp', 'base_attack', 'base_def', 'base_sp_attack', 'base_sp_def', 'base_speed', 'type_1_x', 'name_y', 'url', 'slot_x', 'type_1_y', 'type.url_x', 'slot_y', 'type_2', 'type.url_y']


In [36]:
#Remove and Rename unwanted columns
poke_merge_2 = df_merge_2[['id_x', 'name_x', 'height', 'weight', 'gender_rate',
                         'type_1_y', 'type_2', 'color', 'shape', 'growth_rate',
                          'base_hp', 'base_attack', 'base_def', 'base_sp_attack', 'base_sp_def', 'base_speed',
                         'name_y', 'habitat', 'catch_rate', 'is_baby', 'is_legendary', 'is_mythical',
                         'standard_pic', 'shiny_pic']]
poke_merge_2.head()

,id_x,name_x,height,weight,gender_rate,type_1_y,type_2,color,shape,growth_rate,...,base_sp_def,base_speed,name_y,habitat,catch_rate,is_baby,is_legendary,is_mythical,standard_pic,shiny_pic
0,1,bulbasaur,7,69,1,grass,poison,green,quadruped,medium-slow,...,65,45,NaN,grassland,45,False,False,False,https://raw.githubusercontent.com/PokeAPI/spri...,https://raw.githubusercontent.com/PokeAPI/spri...
1,2,ivysaur,10,130,1,grass,poison,green,quadruped,medium-slow,...,80,60,bulbasaur,grassland,45,False,False,False,https://raw.githubusercontent.com/PokeAPI/spri...,https://raw.githubusercontent.com/PokeAPI/spri...
2,3,venusaur,20,1000,1,grass,poison,green,quadruped,medium-slow,...,100,80,ivysaur,grassland,45,False,False,False,https://raw.githubusercontent.com/PokeAPI/spri...,https://raw.githubusercontent.com/PokeAPI/spri...
3,4,charmander,6,85,1,fire,NaN,red,upright,medium-slow,...,50,65,NaN,mountain,45,False,False,False,https://raw.githubusercontent.com/PokeAPI/spri...,https://raw.githubusercontent.com/PokeAPI/spri...
4,5,charmeleon,11,190,1,fire,NaN,red,upright,medium-slow,...,65,80,charmander,mountain,45,False,False,False,https://raw.githubusercontent.com/PokeAPI/spri...,https://raw.githubusercontent.com/PokeAPI/spri...


In [37]:
#Remove and Rename unwanted columns
df_poke_named = poke_merge_2.rename(columns={'id_x' : 'poke_id', 'name_x' : 'name', 'height': 'height', 'weight' : 'weight', 'gender_rate': 'gender_rate',
                         'type_1_y' : 'type_1', 'type_2' : 'type_2', 'color' : 'color', 'shape' : 'shape', 'growth_rate' : 'growth_rate',
                          'base_hp' :'base_hp', 'base_attack':'base_attack', 'base_def':'base_def', 'base_sp_attack' : 'base_sp_attack', 'base_sp_def' : 'base_sp_def', 'base_speed' : 'base_speed',
                         'name_y' : 'evolves_from', 'habitat': 'habitat', 'catch_rate' : 'catch_rate', 'is_baby' : 'is_baby', 'is_legendary' : 'is_legendary', 'is_mythical' : 'is_mythical',
                         'standard_pic' : 'standard_pic' , 'shiny_pic' : 'shiny_pic'})
df_poke_named.head(10)

,poke_id,name,height,weight,gender_rate,type_1,type_2,color,shape,growth_rate,...,base_sp_def,base_speed,evolves_from,habitat,catch_rate,is_baby,is_legendary,is_mythical,standard_pic,shiny_pic
0,1,bulbasaur,7,69,1,grass,poison,green,quadruped,medium-slow,...,65,45,NaN,grassland,45,False,False,False,https://raw.githubusercontent.com/PokeAPI/spri...,https://raw.githubusercontent.com/PokeAPI/spri...
1,2,ivysaur,10,130,1,grass,poison,green,quadruped,medium-slow,...,80,60,bulbasaur,grassland,45,False,False,False,https://raw.githubusercontent.com/PokeAPI/spri...,https://raw.githubusercontent.com/PokeAPI/spri...
2,3,venusaur,20,1000,1,grass,poison,green,quadruped,medium-slow,...,100,80,ivysaur,grassland,45,False,False,False,https://raw.githubusercontent.com/PokeAPI/spri...,https://raw.githubusercontent.com/PokeAPI/spri...
3,4,charmander,6,85,1,fire,NaN,red,upright,medium-slow,...,50,65,NaN,mountain,45,False,False,False,https://raw.githubusercontent.com/PokeAPI/spri...,https://raw.githubusercontent.com/PokeAPI/spri...
4,5,charmeleon,11,190,1,fire,NaN,red,upright,medium-slow,...,65,80,charmander,mountain,45,False,False,False,https://raw.githubusercontent.com/PokeAPI/spri...,https://raw.githubusercontent.com/PokeAPI/spri...
5,6,charizard,17,905,1,fire,flying,red,upright,medium-slow,...,85,100,charmeleon,mountain,45,False,False,False,https://raw.githubusercontent.com/PokeAPI/spri...,https://raw.githubusercontent.com/PokeAPI/spri...
6,7,squirtle,5,90,1,water,NaN,blue,upright,medium-slow,...,64,43,NaN,waters-edge,45,False,False,False,https://raw.githubusercontent.com/PokeAPI/spri...,https://raw.githubusercontent.com/PokeAPI/spri...
7,8,wartortle,10,225,1,water,NaN,blue,upright,medium-slow,...,80,58,squirtle,waters-edge,45,False,False,False,https://raw.githubusercontent.com/PokeAPI/spri...,https://raw.githubusercontent.com/PokeAPI/spri...
8,9,blastoise,16,855,1,water,NaN,blue,upright,medium-slow,...,105,78,wartortle,waters-edge,45,False,False,False,https://raw.githubusercontent.com/PokeAPI/spri...,https://raw.githubusercontent.com/PokeAPI/spri...
9,10,caterpie,3,29,4,bug,NaN,green,armor,medium,...,20,45,NaN,forest,255,False,False,False,https://raw.githubusercontent.com/PokeAPI/spri...,https://raw.githubusercontent.com/PokeAPI/spri...


In [38]:
#Remove Null Values
df_poke_named["type_2"].fillna("None",inplace=True)
df_poke_named["evolves_from"].fillna("Base",inplace=True)
df_poke_named.head(10)

,poke_id,name,height,weight,gender_rate,type_1,type_2,color,shape,growth_rate,...,base_sp_def,base_speed,evolves_from,habitat,catch_rate,is_baby,is_legendary,is_mythical,standard_pic,shiny_pic
0,1,bulbasaur,7,69,1,grass,poison,green,quadruped,medium-slow,...,65,45,Base,grassland,45,False,False,False,https://raw.githubusercontent.com/PokeAPI/spri...,https://raw.githubusercontent.com/PokeAPI/spri...
1,2,ivysaur,10,130,1,grass,poison,green,quadruped,medium-slow,...,80,60,bulbasaur,grassland,45,False,False,False,https://raw.githubusercontent.com/PokeAPI/spri...,https://raw.githubusercontent.com/PokeAPI/spri...
2,3,venusaur,20,1000,1,grass,poison,green,quadruped,medium-slow,...,100,80,ivysaur,grassland,45,False,False,False,https://raw.githubusercontent.com/PokeAPI/spri...,https://raw.githubusercontent.com/PokeAPI/spri...
3,4,charmander,6,85,1,fire,None,red,upright,medium-slow,...,50,65,Base,mountain,45,False,False,False,https://raw.githubusercontent.com/PokeAPI/spri...,https://raw.githubusercontent.com/PokeAPI/spri...
4,5,charmeleon,11,190,1,fire,None,red,upright,medium-slow,...,65,80,charmander,mountain,45,False,False,False,https://raw.githubusercontent.com/PokeAPI/spri...,https://raw.githubusercontent.com/PokeAPI/spri...
5,6,charizard,17,905,1,fire,flying,red,upright,medium-slow,...,85,100,charmeleon,mountain,45,False,False,False,https://raw.githubusercontent.com/PokeAPI/spri...,https://raw.githubusercontent.com/PokeAPI/spri...
6,7,squirtle,5,90,1,water,None,blue,upright,medium-slow,...,64,43,Base,waters-edge,45,False,False,False,https://raw.githubusercontent.com/PokeAPI/spri...,https://raw.githubusercontent.com/PokeAPI/spri...
7,8,wartortle,10,225,1,water,None,blue,upright,medium-slow,...,80,58,squirtle,waters-edge,45,False,False,False,https://raw.githubusercontent.com/PokeAPI/spri...,https://raw.githubusercontent.com/PokeAPI/spri...
8,9,blastoise,16,855,1,water,None,blue,upright,medium-slow,...,105,78,wartortle,waters-edge,45,False,False,False,https://raw.githubusercontent.com/PokeAPI/spri...,https://raw.githubusercontent.com/PokeAPI/spri...
9,10,caterpie,3,29,4,bug,None,green,armor,medium,...,20,45,Base,forest,255,False,False,False,https://raw.githubusercontent.com/PokeAPI/spri...,https://raw.githubusercontent.com/PokeAPI/spri...


In [39]:
print(df_poke_named.columns.tolist())

['poke_id', 'name', 'height', 'weight', 'gender_rate', 'type_1', 'type_2', 'color', 'shape', 'growth_rate', 'base_hp', 'base_attack', 'base_def', 'base_sp_attack', 'base_sp_def', 'base_speed', 'evolves_from', 'habitat', 'catch_rate', 'is_baby', 'is_legendary', 'is_mythical', 'standard_pic', 'shiny_pic']


In [40]:
#Create Gender Rates for each gender 

gender_dict = {
    "id": poke_id,
    "male_rate" : poke_gender,
    "female_rate" : poke_gender,
    "gender_neutral_rate" : poke_gender,
}

poke_gender_df = pd.DataFrame(gender_dict)
poke_gender_df['male_rate'] = poke_gender_df['male_rate'].replace([0,1,2,3,4,5,6,7,8],[100,87.5,75,62.5,50,37.5,25,12.5,0])
poke_gender_df['female_rate'] = poke_gender_df['female_rate'].replace([1,2,3,4,5,6,7,8],[12.5,25,37.5,50,62.5,75,87.5,100])
poke_gender_df['gender_neutral_rate'] = poke_gender_df['gender_neutral_rate'].replace([-1,1,2,3,4,5,6,7,8],[100,0,0,0,0,0,0,0,0])

poke_gender_df.head(1000)

,id,male_rate,female_rate,gender_neutral_rate
0,1,87.5,12.5,0
1,2,87.5,12.5,0
2,3,87.5,12.5,0
3,4,87.5,12.5,0
4,5,87.5,12.5,0
...,...,...,...,...
992,996,50.0,50.0,0
993,997,50.0,50.0,0
994,998,50.0,50.0,0
995,999,-1.0,-1.0,100


In [41]:
#Merge the tables with the corrected gender
gender_merge_df = pd.merge(df_poke_named,poke_gender_df,left_index=True, right_index=True)
print(gender_merge_df.columns.tolist())

['poke_id', 'name', 'height', 'weight', 'gender_rate', 'type_1', 'type_2', 'color', 'shape', 'growth_rate', 'base_hp', 'base_attack', 'base_def', 'base_sp_attack', 'base_sp_def', 'base_speed', 'evolves_from', 'habitat', 'catch_rate', 'is_baby', 'is_legendary', 'is_mythical', 'standard_pic', 'shiny_pic', 'id', 'male_rate', 'female_rate', 'gender_neutral_rate']


In [42]:
#Reorder and Rename 
final_poke_df = gender_merge_df[['poke_id', 'name', 'height', 'weight', 'male_rate', 'female_rate', 'gender_neutral_rate', 'type_1', 'type_2', 'color', 'shape', 'growth_rate', 'base_hp', 'base_attack', 'base_def', 'base_sp_attack', 'base_sp_def', 'base_speed', 'evolves_from', 'habitat', 'catch_rate', 'is_baby', 'is_legendary', 'is_mythical', 'standard_pic', 'shiny_pic']]
final_poke_df.head()

,poke_id,name,height,weight,male_rate,female_rate,gender_neutral_rate,type_1,type_2,color,...,base_sp_def,base_speed,evolves_from,habitat,catch_rate,is_baby,is_legendary,is_mythical,standard_pic,shiny_pic
0,1,bulbasaur,7,69,87.5,12.5,0,grass,poison,green,...,65,45,Base,grassland,45,False,False,False,https://raw.githubusercontent.com/PokeAPI/spri...,https://raw.githubusercontent.com/PokeAPI/spri...
1,2,ivysaur,10,130,87.5,12.5,0,grass,poison,green,...,80,60,bulbasaur,grassland,45,False,False,False,https://raw.githubusercontent.com/PokeAPI/spri...,https://raw.githubusercontent.com/PokeAPI/spri...
2,3,venusaur,20,1000,87.5,12.5,0,grass,poison,green,...,100,80,ivysaur,grassland,45,False,False,False,https://raw.githubusercontent.com/PokeAPI/spri...,https://raw.githubusercontent.com/PokeAPI/spri...
3,4,charmander,6,85,87.5,12.5,0,fire,None,red,...,50,65,Base,mountain,45,False,False,False,https://raw.githubusercontent.com/PokeAPI/spri...,https://raw.githubusercontent.com/PokeAPI/spri...
4,5,charmeleon,11,190,87.5,12.5,0,fire,None,red,...,65,80,charmander,mountain,45,False,False,False,https://raw.githubusercontent.com/PokeAPI/spri...,https://raw.githubusercontent.com/PokeAPI/spri...


In [43]:
final_poke_df.isnull().any()

poke_id                False
name                   False
height                 False
weight                 False
male_rate              False
female_rate            False
gender_neutral_rate    False
type_1                 False
type_2                 False
color                  False
shape                  False
growth_rate            False
base_hp                False
base_attack            False
base_def               False
base_sp_attack         False
base_sp_def            False
base_speed             False
evolves_from           False
habitat                False
catch_rate             False
is_baby                False
is_legendary           False
is_mythical            False
standard_pic           False
shiny_pic               True
dtype: bool

In [44]:
final_poke_df['shiny_pic'].isnull().sum()

92

In [45]:
gr_lst = []
ps_lst = []
base_url = "https://pokeapi.co/api/v2/growth-rate/"

for p in range(6):
    url = base_url + str(p+1)
    #print(url)
    response = requests.get(url).json()
    
    for specie in response["pokemon_species"]:
        gr_lst.append(response["name"])
        ps_lst.append(specie["name"])

my_dict = {"growth_rate":gr_lst,
           "species_name":ps_lst}

growth_rate_species = pd.DataFrame(my_dict)
growth_rate_species

,growth_rate,species_name
0,slow,growlithe
1,slow,tentacool
2,slow,shellder
3,slow,exeggcute
4,slow,rhyhorn
...,...,...
1005,fast-then-very-slow,swalot
1006,fast-then-very-slow,wailord
1007,fast-then-very-slow,crawdaunt
1008,fast-then-very-slow,drifloon


In [46]:
isinstance(growth_rate_species, pd.DataFrame)

True

In [47]:
# Extract Level & Experience 
lvl_lst = []
exp_lst = []
grth_lst = []
base_url = "https://pokeapi.co/api/v2/growth-rate/"

for p in range(6):
    url = base_url + str(p+1)
    #print(url)
    response = requests.get(url).json()
    
    for level in response["levels"]:
        grth_lst.append(response["name"])
        lvl_lst.append(level["level"])
        exp_lst.append(level["experience"])
        

my_dict = {"growth_rate" :grth_lst,
           "levels":lvl_lst,
           "exp":exp_lst}

growth_rate_levels = pd.DataFrame(my_dict)
growth_rate_levels

,growth_rate,levels,exp
0,slow,1,0
1,slow,2,10
2,slow,3,33
3,slow,4,80
4,slow,5,156
...,...,...,...
595,fast-then-very-slow,96,1415577
596,fast-then-very-slow,97,1460276
597,fast-then-very-slow,98,1524731
598,fast-then-very-slow,99,1571884


## LOADING DATA INTO DATABASE

In [114]:
protocol = 'mysql+pymysql'
username = 'root'
password = pw
host = 'localhost'
port = 3306
database_name = 'pandachams_db'
rds_connection_string = f'{protocol}://{username}:{password}@{host}:{port}/{database_name}'
engine = create_engine(rds_connection_string)

Base = declarative_base()

In [115]:
# Check for existing tables before creation
engine.table_names()

OperationalError: (pymysql.err.OperationalError) (1045, "Access denied for user 'root'@'localhost' (using password: YES)")
(Background on this error at: https://sqlalche.me/e/14/e3q8)

In [94]:
print(final_poke_df.columns.tolist())

['poke_id', 'name', 'height', 'weight', 'male_rate', 'female_rate', 'gender_neutral_rate', 'type_1', 'type_2', 'color', 'shape', 'growth_rate', 'base_hp', 'base_attack', 'base_def', 'base_sp_attack', 'base_sp_def', 'base_speed', 'evolves_from', 'habitat', 'catch_rate', 'is_baby', 'is_legendary', 'is_mythical', 'standard_pic', 'shiny_pic']


In [95]:
final_poke_df.dtypes

poke_id                  int64
name                    object
height                   int64
weight                   int64
male_rate              float64
female_rate            float64
gender_neutral_rate      int64
type_1                  object
type_2                  object
color                   object
shape                   object
growth_rate             object
base_hp                  int64
base_attack              int64
base_def                 int64
base_sp_attack           int64
base_sp_def              int64
base_speed               int64
evolves_from            object
habitat                 object
catch_rate               int64
is_baby                   bool
is_legendary              bool
is_mythical               bool
standard_pic            object
shiny_pic               object
dtype: object

In [96]:
# Creating poke table
class poke(Base):
    extend_existing=True
    __tablename__ = "poke"
    
    poke_id = Column("poke_id", Integer, primary_key = True)
    name = Column("name", String(200))
    height = Column("height", Integer)
    weight = Column("weight", Integer)
    male_rate = Column("male_rate", Float)
    female_rate = Column("female_rate", Float)
    gender_neutral_rate = Column("gender_neutral_rate", Integer)
    type_1 = Column("type_1", String(200))
    type_2 = Column("type_2", String(200))
    color = Column("color", String(200))
    shape = Column("shape", String(200))
    growth_rate = Column("growth_rate", String(200))
    base_hp = Column("base_hp", Integer)
    base_attack = Column("base_attack", Integer)
    base_def = Column("base_def", Integer)
    base_sp_attack = Column("base_sp_attack", Integer)
    base_sp_def = Column("base_sp_def", Integer)
    base_speed = Column("base_speed", Integer)
    evolves_from = Column("evolves_from", String(200))
    habitat = Column("habitat", String(200))
    catch_rate = Column("catch_rate", Integer)
    is_baby = Column("is_baby", Boolean)
    is_legendary = Column("is_legendary", Boolean)
    is_mythical = Column("is_mythical", Boolean)
    standard_pic = Column("standard_pic", String(200))
    shiny_pic = Column("shiny_pic", String(200))

In [97]:
growth_rate_species.dtypes

growth_rate     object
species_name    object
dtype: object

In [98]:
# Creating growth_rate_species table
class poke(Base):
    __tablename__ = "growth_rate_species"
    extend_existing=True

    id = Column("id", Integer, primary_key = True, autoincrement = True)
    growth_rate = Column("growth_rate", String(200))
    species_name = Column("species_name", String(200))

In [99]:
growth_rate_levels.dtypes

growth_rate    object
levels          int64
exp             int64
dtype: object

In [100]:
# Creating growth_rate_levels table
class poke(Base):
    extend_existing=True
    __tablename__ = "growth_rate_levels"
    
    id = Column("id", Integer, primary_key = True, autoincrement = True)
    growth_rate = Column("growth_rate", String(200))
    levels = Column("levels", Integer)
    exp = Column("exp", Integer)

In [101]:
Base.metadata.create_all(bind = engine)

In [102]:
# Checking for existing tables after creation
engine.table_names()

['growth_rate_levels', 'growth_rate_species', 'poke']

In [103]:
final_poke_df.to_sql(name='poke', con=engine, if_exists='append', index=False)

997

In [104]:
growth_rate_species.to_sql(name='growth_rate_species', con=engine, if_exists='append', index=False)

1010

In [105]:
growth_rate_levels.to_sql(name='growth_rate_levels', con=engine, if_exists='append', index=False)

600

In [106]:
pd.read_sql_query('select * from growth_rate_species', con=engine).head()

,id,growth_rate,species_name
0,1,slow,growlithe
1,2,slow,tentacool
2,3,slow,shellder
3,4,slow,exeggcute
4,5,slow,rhyhorn


In [107]:
pd.read_sql_query('select * from growth_rate_levels', con=engine).head()

,id,growth_rate,levels,exp
0,1,slow,1,0
1,2,slow,2,10
2,3,slow,3,33
3,4,slow,4,80
4,5,slow,5,156
